<a href="https://colab.research.google.com/github/carlosgarcianter/duende_programador/blob/main/create_typed_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
TABLE_NAME = "ALCONREPO_REVAL"

In [2]:
matched_fields = {
"Grupo_Cruce": "r.s_grupo_cruce",
"Codigo_Producto": "r.s_codigo_producto",
"Importe": "r.f_importe",
"Codigo_Importe": "r.s_codigo_importe",
"DC2_Financing": "r.f_dc2_financing",
"DC3_Efecto_FX": "r.f_dc3_efecto_fx",
"DC4_Tipo_Cambio": "r.f_dc4_tipo_cambio",
"DC7_Intimas": "r.f_dc7_intimas",
"DE1_Ajustes_7250": "r.f_de1_ajustes_7250",
"DE2_Ajustes_Masivos": "r.f_de2_ajustes_masivos",
"DE3_Otros": "r.i_de3_otros",
"PL_Resultado_Contable": "r.f_pl_resultado_contable",
"PL_Mes_EUR": "r.f_pl_mes_eur",
"PL_Cajas_Mes_EUR": "r.f_pl_cajas_mes_eur",
"PL_Valoracion_Mes_EUR": "r.f_pl_valoracion_mes_eur",
"Resultado_Sigma_Perimetro_Comparable": "r.f_resultado_sigma_perimetro_comparable",
"Resultado_Sigma_Perimetro_Publicable": "r.f_resultado_sigma_perimetro_publicable",
"Resultado_Aplicaciones_BO": "r.f_resultado_aplicaciones_bo",
"Resultado_Contable_Publicado": "r.f_resultado_contable_publicado",
"DC_Diferencias_de_Criterio": "r.f_dc_diferencias_de_criterio",
"DE_Diferencias_Explicadas": "r.f_de_diferencias_explicadas",
"DC1_Diferencia_de_Perimetro": "r.f_dc1_diferencia_de_perimetro",
"Apuntes": "r.i_apuntes",
"Ajustes_Manuales_7250_Signo": "r.f_ajustes_manuales_7250_signo",
"Ajustes_Masivos": "r.f_ajustes_masivos",
"Ajustes_Manuales_7250_BOCO": "r.f_ajustes_manuales_7250_boco",
"Ajustes_Manuales_7250_Recla": "r.f_ajustes_manuales_7250_recla",
"Ajustes_Manuales_7250_Resto": "r.f_ajustes_manuales_7250_resto",
"Ajustes_Masivos_BOCO": "r.f_ajustes_masivos_boco",
"Ajustes_Masivos_Recla": "r.f_ajustes_masivos_recla",
"Ajustes_Masivos_Resto": "r.f_ajustes_masivos_resto",
"Divisa_Importe": "r.s_divisa_importe",
# "Comentario_Pencru": "REPLACE(REPLACE(r.s_Comentario_Pencru, '\\r', ' '), '\\n', ' ') AS s_Comentario_Pencru",
"Comentario_Pencru": "r.s_Comentario_Pencru",
# "Etiquetas_Pencru": "REPLACE(REPLACE(r.s_Etiquetas_Pencru, '\\r', ' '), '\\n', ' ') AS s_etiquetas_Pencru",
"Etiquetas_Pencru": "r.s_Etiquetas_Pencru",
"Divisa_Origen": "r.s_divisa_origen",
"Clasificacion_Ajuste": "r.s_clasificacion_ajuste",
"Numfront_Tratado": "r.s_numfront_tratado",
"Importe_Apunte": "r.f_importe_apunte",
"Cod_Apl_Origen": "r.s_cod_apl_origen",
"Numfront": "r.s_numfront",
"ISIN": "r.s_isin",
"Resultado_BO_Revaluacion": "r.f_resultado_bo_revaluacion",
"DC13_DOP_SLA": "r.f_dc13_dop_sla",
"Resultado_Sigma_Perimetro_Comparable_Revaluacion": "r.f_resultado_sigma_perimetro_comparable_revaluacion",
"Cartera_Mes_Actual": "r.s_cartera_mes_actual",
"Cartera_Mes_Anterior": "r.s_cartera_mes_anterior",
"Estructura": "r.s_estructura",
"Fecha_Vencimiento": "r.d_fecha_vencimiento",
"Fecha_Valor": "r.s_fecha_valor",
"Contrapartida": "r.s_contrapartida",
}

left_fields = {
"Producto_Gestion": "r.rl_s_producto_gestion",
"Marca_Publicable": "r.rl_s_marca_publicable",
"Marca_Contable": "r.rl_s_marca_contable",
"Marca_Autocartera": "r.rl_s_marca_autocartera",
"Portfolio": "r.rl_s_portfolio",
"Instrumento": "r.rl_s_instrumento",
"Oficina_Unidad": "r.rl_s_oficina_unidad",
"Oficina_CPTY": "r.rl_s_oficina_cpty",
"Comentario_Ajuste_GE": "r.rl_s_comentario_ajuste_ge",
"Agrupa": "r.rl_s_agrupa",
}

right_fields = {
"Tipo_BOCO": "r.rr_s_tipo_boco",
"Tipologia_BOCO": "r.rr_s_tipologia_boco",
"Tipo_Procedencia": "r.rr_s_tipo_procedencia",
"Oficina": "r.rr_s_oficina",
"Tipo_Contrato": "r.rr_s_tipo_contrato",
"Contrato": "r.rr_s_contrato",
"Cuenta": "r.rr_s_cuenta",
"Serv_Creador": "r.rr_s_serv_creador",
"Clasificacion_Cuenta": "r.rr_s_clasificacion_cuenta",
"Numback": "r.rr_s_numback",
"Fecha_Contratacion": "r.rr_d_fecha_contratacion",
}

calculated_fields = {
"Diferencias_No_Explicadas": "r.cf_f_diferencias_no_explicadas",
"Marca_Transitorias": "r.cf_s_marca_transitorias",
"Marca_FOBO": "r.cf_s_marca_fobo",
"Operacion_Masa": "r.cf_s_operacion_masa",
"Resultado_BO_ROF": "r.cf_f_resultado_bo_rof",
"Resultado_Sigma_Perimetro_Comparable_ROF": "r.cf_f_resultado_sigma_perimetro_comparable_rof",
}


In [3]:
common_fields = {
  "Id":                   "r.url_id",
  "Process_Code":         "r.process_id",
  "Run_Id":               "CAST(CAST(r.run_number AS FLOAT) AS INTEGER) AS Run_ID",
  "Roll_Up_Relationship": "r.roll_up_relationship",
  "Workflow":             "e.workflow",
  "Status":               "r.status",
  "Group":                "e.group_id",
  "Assigned_To":          "e.assigned_to_user_id",
  "Ageing_Started_At":    "e.creation_timestamp",
  "Age_As_At_Export":     "CAST(e.age AS FLOAT) as Age_as_at_export",
  "Resolved_By":          "e.resolved_by_user_id",
  "Latest_Activity        ": "'' AS Latest_Activity",
  "Score":                "r.score",
  "Input":                "r.input",
  "Labels":               "l.labels AS Labels",
}

common_fields_clean = {
  "Id":                   "STRING",
  "Process_Code":         "STRING",
  "Run_Id":               "INT64",
  "Roll_Up_Relationship": "STRING",
  "Workflow":             "STRING",
  "Status":               "STRING",
  "Group":                "STRING",
  "Assigned_To":          "STRING",
  "Ageing_Started_At":    "DATETIME",
  "Age_As_At_Export":     "FLOAT64",
  "Resolved_By":          "STRING",
  "Latest_Activity":      "STRING",
  "Score":                "FLOAT64",
  "Input":                "STRING",
  "Labels":               "STRING",
}

In [4]:
last_common_fields = {
  "Fecha_ejecucion":  "r.update_timestamp AS Fecha_ejecucion",
}

last_common_clean = {
    "Fecha_Ejecucion":  "DATETIME",
}

In [5]:


matched_fields_clean = {}
for k, v in matched_fields.items():
    new_v = v
    new_v = new_v.replace("r.", "")
    matched_fields_clean[k] = new_v

left_fields_clean = {}
for k, v in left_fields.items():
    new_v = v
    new_v = new_v.replace("r.rl_", "")
    left_fields_clean[k] = new_v

right_fields_clean = {}
for k, v in right_fields.items():
    new_v = v
    new_v = new_v.replace("r.rr_", "")
    right_fields_clean[k] = new_v

calculated_fields_clean = {}
for k, v in calculated_fields.items():
    new_v = v
    new_v = new_v.replace("r.cf_", "")
    calculated_fields_clean[k] = new_v

rules = {
    "b_": "BOOLEAN",
    "d_": "DATE",
    "dt_": "DATETIME",
    "f_": "FLOAT64",
    "i_": "INT64",
    "s_": "STRING",
    "t_": "TIME",
}

typed_recon_fields = {}
for k, v in (matched_fields_clean|left_fields_clean|right_fields_clean|calculated_fields_clean).items():
    tipo = None
    for prefix, t in rules.items():
        if v.startswith(prefix):
            tipo = t
            break
    typed_recon_fields[k] = tipo if tipo else "UNKNOWN"  # fallback por si no hay regla

rename_fields = {}
for k, v in typed_recon_fields.items():
    if v == "FLOAT64":
        new_key = f"{k}"
    elif v == "INT64":
        new_key = f"{k}"
    else:
        new_key = k
    rename_fields[new_key] = v

params = ",\n\t".join(f"`{k}` {v}" for k, v in (common_fields_clean|rename_fields|last_common_clean).items())

third_query = f"""
  CREATE TABLE `dev-bbva-duco-dashboard.reconciliations.{TABLE_NAME}` (
    \t{params}
  )
"""


#===============================================================================

query = third_query
print(query)


  CREATE TABLE `dev-bbva-duco-dashboard.reconciliations.ALCONREPO_REVAL` (
    	`Id` STRING,
	`Process_Code` STRING,
	`Run_Id` INT64,
	`Roll_Up_Relationship` STRING,
	`Workflow` STRING,
	`Status` STRING,
	`Group` STRING,
	`Assigned_To` STRING,
	`Ageing_Started_At` DATETIME,
	`Age_As_At_Export` FLOAT64,
	`Resolved_By` STRING,
	`Latest_Activity` STRING,
	`Score` FLOAT64,
	`Input` STRING,
	`Labels` STRING,
	`Grupo_Cruce` STRING,
	`Codigo_Producto` STRING,
	`Importe` FLOAT64,
	`Codigo_Importe` STRING,
	`DC2_Financing` FLOAT64,
	`DC3_Efecto_FX` FLOAT64,
	`DC4_Tipo_Cambio` FLOAT64,
	`DC7_Intimas` FLOAT64,
	`DE1_Ajustes_7250` FLOAT64,
	`DE2_Ajustes_Masivos` FLOAT64,
	`DE3_Otros` INT64,
	`PL_Resultado_Contable` FLOAT64,
	`PL_Mes_EUR` FLOAT64,
	`PL_Cajas_Mes_EUR` FLOAT64,
	`PL_Valoracion_Mes_EUR` FLOAT64,
	`Resultado_Sigma_Perimetro_Comparable` FLOAT64,
	`Resultado_Sigma_Perimetro_Publicable` FLOAT64,
	`Resultado_Aplicaciones_BO` FLOAT64,
	`Resultado_Contable_Publicado` FLOAT64,
	`DC_Diferenci